# AI(2A) - Human Robot Interaction Project
# 👩‍⚕️  Nurse Robot

## Introduction
This script implements a Spoke Dialog System (SDS) based on Spoken Natural Language Dialog in the context of an automated home nurse assistant, to be used by people with limited capabilities. It simulates actions such as reaching things, provide meals and inspect patient status. The system acquires a spoken input from the user through Google Speech Recognition API. Then, recognizes the intent of the sentence using RASA NLU and retrieves and action result that is transmitted to the user through VocalWare Text-To-Speech API. The order of the sentences, and the recognition of the utterances and dependencies is processed by the libraries spaCy and Keras, building a model that returns the adequate actions given a series of inputs. 

![title](images/NurseRobotMindmap.png)

In [2]:
!which python

/cygdrive/d/ProgramFiles/Anaconda3/envs/bot/python


In [1]:
#imports for project

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import rasa_core
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter
from rasa_nlu.model import Metadata, Interpreter
from rasa_core.utils import EndpointConfig
from rasa_core.run import serve_application
from rasa_nlu.model import Trainer
from rasa_nlu.training_data import load_data
from rasa_core import config
from IPython.display import IFrame
import pandas as pd
import ruamel
import urllib.request
from urllib.request import urlopen
from playsound import playsound
import uuid

import os

import spacy

import warnings
warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #To ignore Tensorflow AVX AVX2 bonary warning

In [4]:
audioFolder = "./audios/"

rooturl = ["https://cache-a.oddcast.com/tts/gen.php?EID=3&LID=1&VID=3&TXT=",
           "&IS_UTF8=1&ACC=3314795&API=2292376&CB=vw_mc.vwCallback&HTTP_ERR=1&vwApiVersion=2"]
characters ={"," : "%2C"," " : "%20","?" : "%3F","'" : "%27"}
def play(phrase):
    for ch in characters:
        phrase = phrase.replace(ch,characters[ch]) #Replace character to match type in URL
    url = rooturl[0] + phrase + rooturl[1] #Build the URL
    file = audioFolder + str(uuid.uuid4()) +'.mp3'
    urllib.request.urlretrieve(url, file)
    playsound(file, True)
    os.remove(file)

In [32]:
play("My name is nurse. How may I help you?")

Some of the data used in this project is not shown in this notebook, but in external files. The content of each relevant file is described hereunder.

* nurse_domain.yml: Defines the actions that can be taken by the agent, the intents that are expected, the entities to be saved if received, and some answers to the basic user utterances.
* stories.md: Contains the examples of dialogs and the specification of the entities that could be found on the phrases.
* config_spacy.json: Saves the pipeline to be used by the NLP library, in this case sklearn with spaCy 


In [30]:
# Declare paths
domain_file = './nurse_domain.yml'
model_path = './models/dialogue'
interpreter_path ='./models/nursebot/interpreter'
training_data_file = './data/stories.md'
conf_file = './config_spacy.json'

In [33]:
# Build the interpreter of the elements of dialog for the agent
!python -m rasa_nlu.train -c config_spacy.json --data data/data.md -o models --fixed_model_name interpreter --project nursebot --verbose

Fitting 2 folds for each of 6 candidates, totalling 12 fits


2019-01-22 16:24:11 INFO     rasa_nlu.utils.spacy_utils  - Trying to load spacy model with name 'en'
2019-01-22 16:24:13 INFO     rasa_nlu.components  - Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
2019-01-22 16:24:14 INFO     rasa_nlu.training_data.loading  - Training data format of data/data.md is md
2019-01-22 16:24:14 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 104 (9 distinct intents)
	- Found intents: 'move', 'thanks', 'goodbye', 'greet', 'attention', 'joke', 'reach', 'affirm', 'deny'
	- entity examples: 29 (2 distinct entities)
	- found entities: 'object', 'place'

2019-01-22 16:24:14 INFO     rasa_nlu.model  - Starting to train component nlp_spacy
2019-01-22 16:24:14 INFO     rasa_nlu.model  - Finished training component.
2019-01-22 16:24:14 INFO     rasa_nlu.model  - Starting to train component tokenizer_spacy
2019-01-22 16:24:14 INFO     rasa_nlu.model  - Finished training component.
2019-01-22 16:24:14 INFO     rasa

In [34]:
action_endpoint = EndpointConfig(url="http://localhost:5055/webhook")
interpreter = RasaNLUInterpreter(interpreter_path)

agent = Agent(domain_file, 
              policies = [MemoizationPolicy(), KerasPolicy(max_history=3, epochs=200, batch_size=50)],
              interpreter=interpreter,
              action_endpoint=action_endpoint)
data = agent.load_data(training_data_file)
agent.train(data)

C:\Users\AndresArciniegas\AppData\Roaming\Python\Python36\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/nursebot/interpreter\entity_synonyms.json'
  "".format(entity_synonyms_file))
Processed Story Blocks: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 120.62it/s, # trackers=20]
Processed actions: 1562it [00:02, 563.15it/s, # examples=1562]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 3, 27)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7680      
_________________________________________________________________
dense (Dense)                (None, 13)                429       
_________________________________________________________________
activation (Activation)      (None, 13)                0         
Total params: 8,109
Trainable params: 8,109
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
744/744 [==============================] - ETA: 17s - loss: 2.5392 - acc: 0.12 - 1s 2ms/step - loss: 2.4448 - acc: 0.3038
Epoch 2/200
744/744 [==============================] - ETA: 0s - loss: 2.3167 - acc: 0.480 - 0s 75us/step - loss: 2.2884 - acc: 0.4395
Epo

Epoch 54/200
744/744 [==============================] - ETA: 0s - loss: 0.1586 - acc: 0.940 - 0s 66us/step - loss: 0.2128 - acc: 0.9489
Epoch 55/200
744/744 [==============================] - ETA: 0s - loss: 0.1679 - acc: 1.000 - ETA: 0s - loss: 0.2002 - acc: 0.952 - 0s 86us/step - loss: 0.1925 - acc: 0.9556
Epoch 56/200
744/744 [==============================] - ETA: 0s - loss: 0.1302 - acc: 0.980 - 0s 66us/step - loss: 0.1976 - acc: 0.9583
Epoch 57/200
744/744 [==============================] - ETA: 0s - loss: 0.1626 - acc: 0.920 - 0s 70us/step - loss: 0.1840 - acc: 0.9583
Epoch 58/200
744/744 [==============================] - ETA: 0s - loss: 0.1403 - acc: 0.980 - 0s 63us/step - loss: 0.1829 - acc: 0.9570
Epoch 59/200
744/744 [==============================] - ETA: 0s - loss: 0.1377 - acc: 0.960 - 0s 66us/step - loss: 0.2068 - acc: 0.9556
Epoch 60/200
744/744 [==============================] - ETA: 0s - loss: 0.1499 - acc: 1.000 - 0s 64us/step - loss: 0.1880 - acc: 0.9530
Epoch 61/2

744/744 [==============================] - ETA: 0s - loss: 0.0623 - acc: 1.000 - 0s 64us/step - loss: 0.0788 - acc: 0.9825
Epoch 114/200
744/744 [==============================] - ETA: 0s - loss: 0.0833 - acc: 0.980 - 0s 66us/step - loss: 0.0517 - acc: 0.9933
Epoch 115/200
744/744 [==============================] - ETA: 0s - loss: 0.0847 - acc: 0.980 - 0s 67us/step - loss: 0.0638 - acc: 0.9852
Epoch 116/200
744/744 [==============================] - ETA: 0s - loss: 0.0735 - acc: 0.980 - 0s 66us/step - loss: 0.0734 - acc: 0.9839
Epoch 117/200
744/744 [==============================] - ETA: 0s - loss: 0.0901 - acc: 0.980 - 0s 68us/step - loss: 0.0616 - acc: 0.9866
Epoch 118/200
744/744 [==============================] - ETA: 0s - loss: 0.0813 - acc: 0.980 - 0s 63us/step - loss: 0.0591 - acc: 0.9919
Epoch 119/200
744/744 [==============================] - ETA: 0s - loss: 0.0052 - acc: 1.000 - 0s 68us/step - loss: 0.0621 - acc: 0.9852
Epoch 120/200
744/744 [==============================] 

Epoch 172/200
744/744 [==============================] - ETA: 0s - loss: 0.0142 - acc: 1.000 - 0s 67us/step - loss: 0.0329 - acc: 0.9946
Epoch 173/200
744/744 [==============================] - ETA: 0s - loss: 0.0951 - acc: 0.980 - 0s 70us/step - loss: 0.0450 - acc: 0.9919
Epoch 174/200
744/744 [==============================] - ETA: 0s - loss: 0.0593 - acc: 1.000 - 0s 64us/step - loss: 0.0225 - acc: 0.9987
Epoch 175/200
744/744 [==============================] - ETA: 0s - loss: 0.0330 - acc: 1.000 - 0s 66us/step - loss: 0.0256 - acc: 0.9960
Epoch 176/200
744/744 [==============================] - ETA: 0s - loss: 0.0223 - acc: 1.000 - 0s 63us/step - loss: 0.0415 - acc: 0.9919
Epoch 177/200
744/744 [==============================] - ETA: 0s - loss: 0.0466 - acc: 0.980 - 0s 68us/step - loss: 0.0317 - acc: 0.9906
Epoch 178/200
744/744 [==============================] - ETA: 0s - loss: 0.0653 - acc: 0.980 - 0s 63us/step - loss: 0.0339 - acc: 0.9933
Epoch 179/200
744/744 [==================

In [35]:
# Save model to be used later
agent.persist(model_path)

In [36]:
# Load agent
agent = Agent.load('./models/dialogue', interpreter=interpreter, action_endpoint=action_endpoint)

In [37]:
# Generate tree of interaction of utterances in possible dialogs
treeFile = "images/dialogtree.html"
agent.visualize("data/stories.md",output_file=treeFile, max_history=2)
IFrame(src=treeFile, width=1000, height=600)

Processed Story Blocks: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 526.93it/s, # trackers=1]


In [38]:
interpreter.parse(u"Pass the remote") # Parsed phrase

{'intent': {'name': 'reach', 'confidence': 0.5784597609657677},
 'entities': [{'start': 9,
   'end': 15,
   'value': 'remote',
   'entity': 'object',
   'confidence': 0.9124695596152568,
   'extractor': 'ner_crf'}],
 'intent_ranking': [{'name': 'reach', 'confidence': 0.5784597609657677},
  {'name': 'move', 'confidence': 0.2852288630345785},
  {'name': 'goodbye', 'confidence': 0.03350030292820014},
  {'name': 'thanks', 'confidence': 0.025830699802019828},
  {'name': 'attention', 'confidence': 0.02313175300330899},
  {'name': 'joke', 'confidence': 0.018132985261360826},
  {'name': 'deny', 'confidence': 0.015794884033042893},
  {'name': 'greet', 'confidence': 0.011291373733584787},
  {'name': 'affirm', 'confidence': 0.008629377238136364}],
 'text': 'Pass the remote'}

In [39]:
# Interpretate a sentence
pp = interpreter.parse(u"Nurse, pass the remote and my book, please") # Parsed phrase
# Print entities present in the sentence
print(pd.DataFrame(pp['entities']))
pd.DataFrame(pp['intent_ranking']).style.apply(lambda x: ['background: lightgreen' if x.name == 0 else '' for i in x],axis=1)

   confidence  end  entity extractor  start   value
0    0.779288   22  object   ner_crf     16  remote
1    0.816153   34  object   ner_crf     30    book


,confidence,name
0,0.5567,reach
1,0.110229,thanks
2,0.0854814,attention
3,0.0671124,goodbye
4,0.0523055,move
5,0.0497657,joke
6,0.0357832,greet
7,0.0244713,deny
8,0.0181513,affirm


In [40]:
# Interpretate a sentence
pp = interpreter.parse(u"Nurse, help me to get to the bathroom please") # Parsed phrase
# Print entities present in the sentence
print(pd.DataFrame(pp['entities']))
pd.DataFrame(pp['intent_ranking']).style.apply(lambda x: ['background: lightgreen' if x.name == 0 else '' for i in x],axis=1)

   confidence  end entity extractor  start     value
0    0.856582   37  place   ner_crf     29  bathroom


,confidence,name
0,0.461307,move
1,0.205685,joke
2,0.122623,reach
3,0.0938955,goodbye
4,0.0380917,thanks
5,0.0294845,attention
6,0.0191963,deny
7,0.0183996,greet
8,0.0113186,affirm


In [46]:
# Interpretate a sentence
pp = interpreter.parse(u"Let's go to the bathroom") # Parsed phrase
# Print entities present in the sentence
print(pd.DataFrame(pp['entities']))
pd.DataFrame(pp['intent_ranking']).style.apply(lambda x: ['background: lightgreen' if x.name == 0 else '' for i in x],axis=1)

   confidence  end entity extractor  start     value
0    0.964441   24  place   ner_crf     16  bathroom


,confidence,name
0,0.88052,move
1,0.0242743,joke
2,0.0226783,goodbye
3,0.0221447,reach
4,0.0164524,thanks
5,0.0112273,deny
6,0.00917437,attention
7,0.00680039,greet
8,0.00672822,affirm


In [42]:
print("[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)\n")
while True:
    print(">> ", end='')
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    if(len(responses) == 0):
        print("Action returned 0 elements. Maybe the action listener is not active. Activate listener:\n python -m rasa_core_sdk.endpoint --actions actions")
    for response in responses:
        print("👩 "+" >> " + response["text"]+"\n")
        play(response["text"])

[INFO] The nurse is ready to listen. Please start a dialog... (Type 'stop' to quit)

>> Hello
👩  >> Greetings. You can call me Nurse. How may I help you?

>> Tell me a joke
👩  >> Chuck Norris can reverse Convection.

>> Tell me another joke
👩  >> Chuck Norris can use bulldozer to knock a mosquito off of a bright red macaque's ass.

>> Bring me my book
👩  >> Of course! There you go.

>> Thank you
👩  >> You're welcome.

👩  >> Do you need anything else?

>> Not for now
👩  >> I'm at your service.

>> stop
